### Fit $Q_{int}$ vs P, T for R82_F12_5p48

In [ ]:
from pathlib import Path
from lmfit import Parameters
import numpy as np
from rrfit.waterfall import fitIterated, Fit_QIntVsTemp, plot_Qi_vs_temp

from betata.resonator_studies.trace import load_fitted_traces, plot_fitted_trace
from betata.resonator_studies.resonator import add_qpt_fit_params, load_resonator, save_resonator

CWD = Path.cwd()

Specify resonator file

In [ ]:
resonator_name = "R82_F12_5p48"
resonator_file = CWD / f"out/resonator_studies/{resonator_name}.h5"

Load resonator data

In [ ]:
resonator = load_resonator(resonator_file)
resonator.traces = load_fitted_traces(resonator_file)

Plot $Q_{int}$ vs P, T

In [ ]:
plot_Qi_vs_temp(resonator)

Inspect specified S21 traces and exclude additional traces if needed

In [ ]:
powers = []
min_temp, max_temp = np.inf, -np.inf
for trace in resonator.traces:
    if trace.power in powers and min_temp < trace.temperature < max_temp:
        plot_fitted_trace(trace, resonator_name)

In [ ]:
traces_to_exclude = []
traces_to_include = []

for trace in resonator.traces:
    if trace.power in [-30]:
        trace.is_excluded = True
    if trace.id in traces_to_exclude:
        trace.is_excluded = True
    if trace.id in traces_to_include:
        trace.is_excluded = False

Perform initial iterated fit

In [ ]:
# fit parameters
init_params = Parameters()
init_params.add("delta_QP0", value=2e-4, min=0)
init_params.add("Q_TLS0", value=1e6, min=0)
init_params.add("tc", value=2.0, min=0.0, max=2.0)
init_params.add("Q_other", value=1e7, min=0)
init_params.add("beta", value=1, min=0, max=2.0)
init_params.add("beta2", value=1, min=0, max=2.0)
init_params.add("D_0", value=100, min=0)

# initial guess bounds
bounds_dict = {}
bounds_dict["Q_TLS0"] = (1e5, 1e7)
bounds_dict["beta2"] = (0, 1.0)
bounds_dict["beta"] = (0, 1.0)
bounds_dict["D_0"] = (1, 100)
bounds_dict["delta_QP0"] = (1e-4, 1e2)
bounds_dict["tc"] = (0.25, 1.0)
bounds_dict["Q_other"] = (1e5, 1e9)

_, _, _, _ = fitIterated(
    resonator, 
    bounds_dict, 
    numIter=100, 
    retries=25, 
    init_params=init_params
)

Perform final self-consistent fit

In [ ]:
fit_params, _, fittedFig_1 = Fit_QIntVsTemp(
    resonator, 
    resonator.best_params, 
    consistent=True
)

In [ ]:
fit_params

Save fit params to resonator hdf5 file

In [ ]:
add_qpt_fit_params(resonator, fit_params)
resonator.qpt_fit_trace_ids = [tr.id for tr in resonator.traces if not tr.is_excluded]
save_resonator(resonator, resonator_file)